In [1]:
import os
import numpy as np
import numpy.linalg as la
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt

In [8]:
from model import Net
from tdnn import TDNN
from dataloader import DataLoader

In [9]:
device = torch.device("cuda:0")
torch.cuda.is_available()

True

In [10]:
file_name = 'mfcc'

train_path = 'mfcc'
y_train = np.load(train_path + '/speakers.npy')
train_size = y_train.shape[0]
print(train_size)

test_path = 'mfcc_test'
y_test = np.load(test_path + '/speakers.npy')
test_size = y_test.shape[0]
print(test_size)

model = torch.load('models/model_acc90.pkl').to(device)

1092008
36237


In [ ]:
embeddings_train = np.zeros((train_size, 512))
for i in range(train_size):
    print(f'\r{i}/{train_size}', end='')
    mfcc = torch.load(train_path + '/' + file_name + str(i)).to(device)
    emb = model.embedding(mfcc).cpu().detach().numpy().squeeze(0)
    embeddings_train[i] = emb
np.save('embeddings_train.npy', embeddings_train)

842037/1092008

In [ ]:
embeddings_test = np.zeros((test_size, 512))
for i in range(test_size):
    print(f'\r{i}/{test_size}', end='')
    mfcc = torch.load(test_path + '/' + file_name + str(i)).to(device)
    emb = model.embedding(mfcc).cpu().detach().numpy().squeeze(0)
    embeddings_test[i] = emb
np.save('embeddings_test.npy', embeddings_test)

In [ ]:
embeddings_train = np.load('embeddings_train.npy')
embeddings_test = np.load('embeddings_test.npy')

### Simple cosine dist

In [ ]:
threshold = 0.75
n_same = 0
k_same = 0
n_diff = 0
k_diff = 0
ITERATIONS = 100000
for i in range(ITERATIONS):
    print(f'\r{i}/{ITERATIONS}', end='')
    rand_idx = np.random.choice(test_size, size=2, replace=True)
    same = (y_test[rand_idx[0]] == y_test[rand_idx[1]])
    pair = embeddings_test[[rand_idx[0], rand_idx[1]]]
    cos_dist = pair[0] @ pair[1] / (la.norm(pair[0]) * la.norm(pair[1]))
    if cos_dist > threshold:
        if same:
            n_same += 1
        else:
            k_diff += 1
            n_diff += 1
    else:
        if same:
            k_same += 1
            n_same += 1
        else:
            n_diff += 1

print(f'Total same pairs: {n_same}\t err rate: {k_same/n_same}')
print(f'Total diff pairs: {n_diff}\t err rate: {k_diff/n_diff}')

### PLDA

In [ ]:
from plda.model import Model
plda = Model(embeddings_train, y_train, n_principal_components=200)
transformed_test = plda.transform(embeddings_test, from_space='D', to_space='U_model')

In [ ]:
threshold = 0
n_same = 0
k_same = 0
n_diff = 0
k_diff = 0
ITERATIONS = 10000
for i in range(ITERATIONS):
    print(f'\r{i}/{ITERATIONS}', end='')
    rand_idx = np.random.choice(test_size, size=2, replace=True)
    same = (y_test[rand_idx[0]] == y_test[rand_idx[1]])
    pair = transformed_test[[rand_idx[0], rand_idx[1]]]
    plda_score = plda.calc_same_diff_log_likelihood_ratio(pair[0], pair[1])
    if plda_score > threshold:
        if same:
            n_same += 1
        else:
            k_diff += 1
            n_diff += 1
    else:
        if same:
            k_same += 1
            n_same += 1
        else:
            n_diff += 1

print(f'Total same pairs: {n_same}\t err rate: {k_same/n_same}')
print(f'Total diff pairs: {n_diff}\t err rate: {k_diff/n_diff}')